<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/02_Normalisation_des_textes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Normalisation des textes

La normalisation d'un texte a pour objectif de diminuer le "bruit" et à amplifier ce que l'on cherche à écouter. Par bruit, on entend les variations linguistiques qui sont rares ou non nécessaires à considérer pour la tâche que l'on souhaite réaliser.

La **normalisation** d'un texte consiste donc à 
- supprimer ce qui est rare ou non porteur de sens et qui apporte du bruit, 
- mettre en avant ce qui est porteur de sens en réduisant/unifiant les variantes.

En pratique cela signifie : supprimer les balises  (e.g. _`<div>`_->_∅_), remplacer les variantes d'un mot par une forme référente par expansion systématique des contractions (e.g. _l'_->_le_) ou par lemmatisation ou stemming (e.g. _la_->_le_), supprimer les caractères qui ne sont pas lettres, supprimer les mots vides (e.g. _le_->_∅_), substituer toutes les occurrences d'hashtags par un unique token `#hashtag`, également pour les urls...

Suivant la tâche, on peut s'intéroger si la normalisation doit aussi **corriger** les mots mal orthographiés ou les répétitions de caractères qu'ils soient alphabétiques ou autres. En effet la répétition d'une même lettre ou d'un caractère de ponctuation peut marquer l'emphase dans un propos e.g. "suuuuuuuuuuuper !!!!!". 

## Installation de l'environnement : chargement des modèles et des données

Nous utiliserons un corpus de romans de Jules Verne de la littérature française. En pratique le corpus est un répertoire, et les fichiers sont des oeuvres distinctes de Jules Verne. Exécuter le code suivant. 

In [ ]:
# get the corpus and install in local
!mkdir data
!wget -nc https://raw.githubusercontent.com/nicolashernandez/teaching_nlp/main/data/JulesVerne.zip -P data
!unzip data/JulesVerne.zip -d data

Dans le cadre de ce travail nous utiliserons un seul roman (fichier) à savoir _20000 Lieues sous les mers_. Le code suivant permet de vérifier le contenu du fichier ainsi que de définir des variables qui seront utilisées par la suite pour illustrer le fonctionnement des différents codes fournis : `novel_name` et `document`.

In [ ]:
# have a look at a sample of one particular file
novel_path = "data/JulesVerne/Jules Verne_20000 Lieues sous les mers.txt"

import regex as re
novel_name = re.search("_(.+)\.txt", novel_path).group(1)
print (novel_name, novel_path)

with open(novel_path, encoding='utf8') as f: 
  document = f.read()
  print (document[:1000])

## Mots vides

### QUESTION 

* Executer le code ci-dessous pour vérifier que la liste de mots vides de nltk pour le français contient bien que des mots vides. Y-a-t-il des mots "non porteur de sens" que vous auriez pensé retrouver ?

In [ ]:
import nltk 
nltk.download('stopwords')
# retrieve a list of french stop words 
fr_stop_words = nltk.corpus.stopwords.words('french')
print ('fr_stop_words:',fr_stop_words)

### VOTRE REPONSE

**TODO**


## Opérations de normalisation

### QUESTION 

La méthode ci-dessous prétraite un document (une chaîne de caractère) en réalisant un certain nombre d'opérations de normalisation. Replacer les commentaires suivant au bon endroit dans le code.

```
    # filter tokens with small length
    # tokenize document
    # filter stopwords out of document
    # substitute all characters which are not a whitespace or a letter by a whitespace
    # to lower case
```



### VOTRE REPONSE

**TODO**



In [ ]:
import nltk

# the native re module does not handle \p{} unicode property 
import regex as re

# define a tokenizer which tokenizes a text by splitting at each whitespace character
ws_tokenizer = nltk.WhitespaceTokenizer()

# normalize a string of text
def normalize_characters(doc):
    # TODO 
    doc = re.sub('[^\p{L}\s]', ' ', doc)

    # TODO
    doc = doc.lower()
  
    # TODO 
    return ws_tokenizer.tokenize(doc)

# normalize tokens
def normalize_tokens(tokens):
    # TODO 
    tokens = [token for token in tokens if token not in fr_stop_words]

    # TODO 
    tokens = [token for token in tokens if len(token) >2]

    # re-create document from filtered tokens
    #doc = ' '.join(tokens)
    #return doc
    return tokens

# all in one
def preprocess_document(document):
  return normalize_tokens(normalize_characters(document))


In [ ]:
# le code suivant permet d'illustrer le fonctionnement de la normalisation
sample_document = "Tu pourras (c'est ouf gr!) même goûter au gin de Marina,et mon rhum arrangé!!! 😜🤝🏩🏣👍"
sample_preprocessed_document = normalize_tokens(normalize_characters(sample_document))
print ('document:', sample_document)
print ('preprocessed_document:', sample_preprocessed_document)

## Taille du texte vs. taille de vocabulaire

### QUESTION

Le code suivant prétraite le document sans et avec filtrage de tokens (notamment les mots vides). Pour chaque résultat de prétraitement, le nombre total de tokens et la taille du vocabulaire sont affichés.

* Comparer les tailles de vocabulaire et le nombre total de tokens avant/après filtrage des mots vides de sens. Que traduisent ces chiffres ? 

In [ ]:
# suppression de certains caractères, passage de tous les caractères en minuscule, tokenization
normalized_characters = normalize_characters(document)
print ('Before filtering - number of tokens:', len(normalized_characters), '; vocabulary size:', len(set(normalized_characters)))

# suppression de certains tokens
normalized_tokens = normalize_tokens(normalized_characters)
print ('After filtering - number of tokens:', len(normalized_tokens), '; vocabulary size:', len(set(normalized_tokens)))

### VOTRE REPONSE

**TODO**


## Vérifier si la loi de Zipf s'applique.

La loi de zipf :

> En classant les mots d’un texte par fréquence décroissante, on observe que la fréquence, `freq(m)`, d’un mot `m` dans un
texte est corrélée à son rang `rang(m)` par une
loi du type : `freq(m) * rang(m) = K` avec `K`
une constante. 
Peu de mots fréquents et grand nombre de
mots rares.

La _fréquence d'un mot_ est son _nombre d'occurrence_ sur la somme totale des occurrences de tous les mots. La fréquence constitue une forme de normalisation numérique. Une tendance observée avec le nombre d'occurrences "devrait" être la même que celle observée avec une fréquence.




### QUESTION : mots vides vs. mots porteurs de sens

Le code ci-dessous traite les mots avant filtrage des mots vides. Il utilise la méthode `FreqDist` d'un paquet de nltk pour calculer la fréquence des tokens du corpus d'un des auteurs, puis affiche les `n` tokens les plus _fréquents_ ; on dit aussi les plus _communs_ (`most_common`). 

* Exécutez-le. Est-ce bien la fréquence qui est observée ?
* Faire varier le nombre de mots les plus fréquents à observer en jouant avec la variable `most_common_n`. Essayer des valeurs de 10 à 1000.  Quand vous regardez les extrêmes (les plus fréquents vs. les moins fréquents des plus fréquents...), que pouvez-vous dire sur la capacité des mots à décrire un contenu (à donner du sens) ?


In [ ]:
# import required libraries 
from nltk.probability import FreqDist

# find the frequency 
fdist = FreqDist(normalized_characters)

most_common_n = 10

# print the n most common 
print ('Les', most_common_n, 'mots les plus fréquents sont :', fdist.most_common(most_common_n))

### VOTRE REPONSE

**TODO**


### QUESTION : observation de la normalisation des mots les plus communs

Le code suivant trace une courbe de la "fréquence" des mots ordonnée sur le rang de la fréquence.

* Dans le code ci-dessous, rajoutez quelques lignes pour visualiser "aussi" la "fréquence" des mots après normalisation des tokens (i.e. après application de la méthode `normalize_tokens`). Trouvez-vous que la méthode `normalize_tokens` a bien fait son travail ?
* Comment caracterisez-vous la 2nd courbe par rapport à la 1ère ?  

In [ ]:
# plot the graph of frequency for n most common  :
import matplotlib.pyplot as plt
most_common_n = 40

fdist = FreqDist(normalized_characters)
print ('Les', most_common_n, 'mots les plus fréquents sont :', fdist.most_common(most_common_n))
fdist.plot(most_common_n)

fdist = FreqDist(normalized_tokens)
print ('Les', most_common_n, 'mots les plus fréquents sont :', fdist.most_common(most_common_n))
fdist.plot(most_common_n)

### VOTRE REPONSE

**TODO**


### QUESTION : observation de la fréquence à grande échelle

Le code ci-dessous fait globalement la même chose que ci-dessus mais il offre davantage de contrôle sur le calcul de la fréquence. Cela permet aussi de jouer plus finement avec le traçage de la courbe (qui ici permet de considérer tous les mots et non seulement les plus communs). 

* Executez le code et visualizez la courbe. Eventuellement commenter/décommenter les lignes qui vont bien pour passer à une échelle logarithmique et plus aisément observer des valeurs très petites. Est-ce que la loi de Zipf s'applique sur ce document ? 
* Remarquez que le code vous permez aussi d'observer la courbe avec le nombre (ou compte) d'occurrences plutôt que la fréquence...

In [ ]:
from collections import Counter
# We use reduce to concatenate all the lists in tokenized_author_data, but we don't use "set" 
# so that we can count occurencies with a Counter object 
# Count the number of occurrences of each token
occurrences_counter = Counter(normalized_characters)

# Formating the counter object to a proper dataset
import pandas as pd
d = pd.DataFrame(occurrences_counter, index=['occurrences']).transpose().reset_index()
d.columns=['word', 'occurrences']

# Computing frequencies instead of occurrences
nb_total = d.occurrences.sum()
d['freq'] = d.occurrences.apply(lambda x: x/nb_total)
# d['freq'] = d.occurences.apply(lambda x: x) # <- nombre d'occurrences

# Sorting by frequency, most frequent word at the top of the df
d = d.sort_values('freq', ascending=False)

# Plot
import matplotlib.pyplot as plt
plt.figure()
plt.grid()
plt.xscale('log')  # Using log scale
plt.yscale('log')  # Using log scale
plt.xlabel('log(rank)')
plt.ylabel('log(frequency)')
#plt.xlabel('rank')
#plt.ylabel('frequency')
#plt.ylabel('count')
plt.title("Zipf law : {0}".format(novel_name))
x = list(range(d.shape[0]))
plt.plot(x, d.freq)
plt.show()

### VOTRE REPONSE

**TODO**

